In [ ]:
# Import dependencies and the parser module
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
from xbrl_parser import XbrlParser

# Set target universe
target_companies = ["ajinomoto", "mufg", "toyota", "honda"]
target_fys = [f"FY{y}" for y in range(2015, 2025)]
current_dir = Path.cwd()

for target_company in target_companies:
    for target_fy in target_fys:
        
        # Define input file
        xbrl_zip_path = current_dir / "input" / "XBRL" / target_company / (target_company+"_"+target_fy+".zip")
        
        # Define output file
        parsed_csv_path = current_dir / "output" / target_company / (target_company+"_"+target_fy+"_parsed_xbrl.csv")
        parsed_csv_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Generate objects
        parser = XbrlParser(xbrl_zip_path)
        df_xbrl = parser.parse()
        
        # Save data
        df_xbrl.to_csv(parsed_csv_path, index=False, encoding='utf-8-sig')

In [ ]:
# Import dependencies and the parser module
%load_ext autoreload
%autoreload 2
from datetime import datetime
from pathlib import Path
from financial_analyzer import FinancialAnalyzer, load_mapping_with_override
from slides_core import (
    SlideConfig,
    PowerPointGeneratorEngine,
    SlideCover,
    SlideDeck,
    SlidePage,
)

def extract_page_columns(page_def: dict) -> list:
    mapping = page_def.get("data_mapping", {})
    category = page_def.get("category")
    cols = []
    if category == "combo_bar_line_2axis":
        cols += [t["col"] for t in mapping.get("bar_traces", [])]
        cols += [t["col"] for t in mapping.get("line_traces", [])]
        cols.append(mapping.get("x_col", "period_label"))
    elif category == "balance_sheet":
        for stack_key in ("left_stack", "left_stack_for_bank", "left_stack_summary", "right_stack", "right_stack_for_bank", "right_stack_summary"):
            cols += [item["col"] for item in mapping.get(stack_key, [])]
        cols.append(mapping.get("total_assets_col", "Total Assets"))
    elif category == "portfolio_timeseries":
        cols += [t["col"] for t in mapping.get("series", [])]
        cols.append(mapping.get("x_col", "period_label"))
    return sorted({col for col in cols if col})

for target_company in target_companies:
    csv_dir = current_dir / "output" / target_company
    csv_files = list(csv_dir.glob("*.csv"))
    dfs = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file.resolve())
        dfs.append(df)
    df_target = pd.concat(dfs)
    standard = None
    if "Standard" in df_target.columns and not df_target["Standard"].dropna().empty:
        standard = df_target["Standard"].dropna().mode().iloc[0]
    mapping = load_mapping_with_override(Path("custom_mapping.json"), standard=standard, company_name=target_company)
    analyzer = FinancialAnalyzer(df_target, mapping=mapping, standard=standard, company_name=target_company)

    df_pl = analyzer.get_pl_data()
    df_bs = analyzer.get_bs_data()
    df_pl.to_csv(f"DUMP_{target_company}_PL.csv", index=None)
    df_bs.to_csv(f"DUMP_{target_company}_BS.csv", index=None)

    data_store, slides_structure = analyzer.build_slide_payload()

    sample_text = {
        "PL": [
            {"title": "Margin", "body": "Operating income trend mirrors revenue, with limited margin expansion."},
            {"title": "Scale", "body": "Latest FY shows revenue recovery while operating income stabilizes."},
        ],
        "BS": [
            {"title": "Leverage", "body": "Equity share improves, but liabilities remain concentrated."},
            {"title": "Liquidity", "body": "Cash and equivalents fluctuate with working-capital swings."},
        ],
        "CF": [
            {"title": "Cash Conversion", "body": "Operating cash flow tracks profit with periodic volatility."},
            {"title": "Funding Mix", "body": "Financing flows offset investment outflows in down years."},
        ],
        "Portfolio": [
            {"title": "Risk Mix", "body": "Equity exposure rises relative to debt; derivatives net stays modest."},
            {"title": "Volatility", "body": "Portfolio size expands in strong years and contracts in tightening cycles."},
        ],
    }

    pages = []
    for slide_def in slides_structure:
        page_def = dict(slide_def)
        source_key = page_def.pop("data_source", None)
        if source_key:
            page_def["data_frame"] = data_store.get(source_key)
        page_def["data_columns"] = extract_page_columns(page_def)
        text_blocks = sample_text.get(page_def.get("slide_title", ""))
        if text_blocks:
            page_def["text_blocks"] = text_blocks
            page_def["proposal_section_title"] = "Key Findings"
        pages.append(SlidePage(**page_def))

    selected_pages = pages

    cover = SlideCover(
        main_title=f"{target_company} Financial Review",
        sub_title="Generated from XBRL",
        date=datetime.now().strftime("%Y-%m-%d"),
    )

    deck = SlideDeck(cover=cover, pages=selected_pages)
    engine = PowerPointGeneratorEngine(SlideConfig())
    engine.generate(
        deck=deck,
        data_store=data_store,
        filename_prefix=f"{target_company}_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    )
